In [187]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import sys
import os

script_path = os.path.abspath("../src/features")
sys.path.append(script_path)
from sat_chunk_normalize import rescale_scores_with_categories, Chunker


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Reading in data and normalizing scores

In [82]:
!ls ../data/processed/sat_train

ielts  sat_train  sat_val


In [54]:
fp = '../data/raw/sat_train/training_set_rel3.xls'

In [55]:
df = pd.read_excel(fp)

In [56]:
df.head(1)

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4.0,4.0,NaN,8.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
# df.groupby(by='essay_set')['domain1_score'].agg(['min', 'max']).reset_index()

In [58]:
# !ls ../src/features/sat_chunk_normalize.py

In [61]:
%reload_ext autoreload
df_rescaled = rescale_scores_with_categories(df=df, score_column='domain1_score', output_range=(1, 10))

In [62]:
df_rescaled

,essay_id,essay_set,essay,rescaled_score,low_med_hi,low_med_hi_numeric
0,1,1,"Dear local newspaper, I think effects computer...",6.40,medium,2
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",7.30,medium,2
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",5.50,medium,2
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",8.20,high,3
4,5,1,"Dear @LOCATION1, I know having computers has a...",6.40,medium,2
...,...,...,...,...,...,...
12973,21626,8,In most stories mothers and daughters are eit...,5.50,medium,2
12974,21628,8,I never understood the meaning laughter is th...,4.96,low,1
12975,21629,8,"When you laugh, is @CAPS5 out of habit, or is ...",6.40,medium,2
12976,21630,8,Trippin' on fen...,6.40,medium,2


In [77]:
df['essay_split_len'] =  df['essay'].str.split(' ').apply(len)

In [79]:
has_line_breaks = df_rescaled['essay'].str.contains('\n', na=False).any()
has_line_breaks

False

In [81]:
df_rescaled.columns

Index(['essay_id', 'essay_set', 'essay', 'rescaled_score', 'low_med_hi',
       'low_med_hi_numeric'],
      dtype='object')

In [83]:
train_full_out_fp = '../data/processed/sat_train/train_full_processed.csv'

In [84]:
df_rescaled.to_csv(train_full_out_fp,index=False)

In [85]:
df = pd.read_csv(train_full_out_fp)

In [86]:
df

,essay_id,essay_set,essay,rescaled_score,low_med_hi,low_med_hi_numeric
0,1,1,"Dear local newspaper, I think effects computer...",6.40,medium,2.0
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",7.30,medium,2.0
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",5.50,medium,2.0
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",8.20,high,3.0
4,5,1,"Dear @LOCATION1, I know having computers has a...",6.40,medium,2.0
...,...,...,...,...,...,...
12973,21626,8,In most stories mothers and daughters are eit...,5.50,medium,2.0
12974,21628,8,I never understood the meaning laughter is th...,4.96,low,1.0
12975,21629,8,"When you laugh, is @CAPS5 out of habit, or is ...",6.40,medium,2.0
12976,21630,8,Trippin' on fen...,6.40,medium,2.0


In [88]:
val_fp = '../data/raw/sat_val/valid_set.xlsx'
df_val = pd.read_excel(val_fp)

In [90]:
df_val.head(2)

,essay_id,essay_set,essay,domain1_predictionid,domain2_predictionid
0,1788,1,"Dear @ORGANIZATION1, @CAPS1 more and more peop...",1788,NaN
1,1789,1,Dear @LOCATION1 Time @CAPS1 me tell you what I...,1789,NaN


In [94]:
test_fp = '../data/raw/sat_test/test_set.tsv'
df_test = pd.read_csv(test_fp, sep='\t', encoding='latin1')  


In [95]:
df_test

,essay_id,essay_set,essay,domain1_predictionid,domain2_predictionid
0,2383,1,I believe that computers have a positive effec...,2383,NaN
1,2384,1,"Dear @CAPS1, I know some problems have came up...",2384,NaN
2,2385,1,"Dear to whom it @MONTH1 concern, Computers are...",2385,NaN
3,2386,1,"Dear @CAPS1 @CAPS2, @CAPS3 has come to my atte...",2386,NaN
4,2387,1,"Dear Local newspaper, I think that people have...",2387,NaN
...,...,...,...,...,...
4249,22238,8,Just One @CAPS1 @CAPS2 can bring people togeth...,25238,NaN
4250,22239,8,@CAPS8 Was my first time going to the band ca...,25239,NaN
4251,22240,8,Laughter is one of the most important things i...,25240,NaN
4252,22241,8,A @CAPS1 Of @C...,25241,NaN


## Splitting the Test Set

In [99]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_rescaled, test_size=0.2, stratify=df_rescaled['essay_set'], random_state=42)


In [106]:
# train['essay_set'].value_counts(normalize=True), test['essay_set'].value_counts(normalize=True)

In [110]:
train_full_out_fp = '../data/processed/sat_train/train_full_processed.csv'
train_full_out_fp

'../data/processed/sat_train/train_full_processed.csv'

In [107]:
train.to_csv(train_full_out_fp,index=False)

In [111]:
test_full_out_fp = '../data/processed/sat_test/test_full_processed.csv'

In [112]:
test.to_csv(test_full_out_fp,index=False)

## Chunking

In [172]:
train.essay.apply(len)

6578     1000
2936     2358
4209      361
10059    1535
7855      409
         ... 
7307       83
11238    1530
8405      736
1866     1899
7692      585
Name: essay, Length: 10382, dtype: int64

In [177]:
train_chunker = Chunker(train)

In [178]:
train_chunker.get_chunks(min_length=100)

,essay_id,essay_set,paragraph,rescaled_score,low_med_hi,low_med_hi_numeric
0,10135,4,The reason why the author concludes the story ...,7.000000,medium,2
1,4131,2,Would you censor the @CAPS1? i @CAPS2 i sure w...,4.600000,low,1
2,4131,2,Throught out the story the @CAPS5 gives the bo...,4.600000,low,1
3,4131,2,Why?! The @CAPS1 for example is just a book fu...,4.600000,low,1
4,4131,2,Thanks to a close friend. @CAPS8 could also be...,4.600000,low,1
...,...,...,...,...,...,...
16520,18459,7,"I got to see @NUM1 giant wolfs, snakes, monkey...",7.545455,high,3
16521,13151,5,The mood created by the author in this memoir ...,5.500000,medium,2
16522,3061,2,"Dear Katherine Paterson, Do you believ...",4.600000,low,1
16523,3061,2,Either way I think that they shouldn't remove ...,4.600000,low,1


In [179]:
train_chunker.chunked_df['paragraph'].str.split(' ').apply(len).min(), train_chunker.chunked_df['paragraph'].str.split(' ').apply(len).max()

(1, 435)

In [180]:
train_chunker.chunked_df['paragraph'].str.split(' ').apply(len).argmin(), train_chunker.chunked_df['paragraph'].str.split(' ').apply(len).argmax()

(4356, 1672)

In [186]:
train_chunker.chunked_df.loc[1672,'paragraph']

'@CAPS5 were all at my party having a great time right @CAPS5 my cousin @PERSON1 says "@CAPS5 who wants to play the game @CAPS4? " @CAPS6 we all say well whats that?" @CAPS6 he goes over all the rules like a said before @CAPS15 we say "lets play." @CAPS5 me @CAPS15 my friend @CAPS9 are running away @CAPS15 all of the sudden we start hearing my cousins @CAPS4 noise that means he coming @CAPS15 he freaked out @CAPS15 ran away @CAPS6 when he leaves you @CAPS14 hear the screeching scream come out @CAPS15 you here his balloon pop that means that he got out @CAPS15 were all @CAPS14 laughing because when he flew he launched about @NUM3 feet backwards it was the funniest thing ever because when i got over there he was still on the ground @CAPS15 everyone @CAPS14 it trying to breathe i swear i think everyone was about to wet them selves @CAPS15 the probably would\'ve laughed some more if it wasn\'t for the game @CAPS15 @CAPS6 @CAPS9 got up @CAPS15 started chasing people with my cousin @PERSON1 

In [184]:
l = '@CAPS5 were all at my party having a great time right @CAPS5 my cousin @PERSON1 says "@CAPS5 who wants to play the game @CAPS4? " @CAPS6 we all say well whats that?" @CAPS6 he goes over all the rules like a said before @CAPS15 we say "lets play." @CAPS5 me @CAPS15 my friend @CAPS9 are running away @CAPS15 all of the sudden we start hearing my cousins @CAPS4 noise that means he coming @CAPS15 he freaked out @CAPS15 ran away @CAPS6 when he leaves you @CAPS14 hear the screeching scream come out @CAPS15 you here his balloon pop that means that he got out @CAPS15 were all @CAPS14 laughing because when he flew he launched about @NUM3 feet backwards it was the funniest thing ever because when i got over there he was still on the ground @CAPS15 everyone @CAPS14 it trying to breathe i swear i think everyone was about to wet them selves @CAPS15 the probably would\'ve laughed some more if it wasn\'t for the game @CAPS15 @CAPS6 @CAPS9 got up @CAPS15 started chasing people with my cousin @PERSON1 @CAPS15 the rest of us @CAPS14 ran out of there as quickly as possible @CAPS15 yet again you here my cousin making his @CAPS4 noise @CAPS15 people running @CAPS15 screaming @CAPS15 my friend make trying to make the same @CAPS4 noise but cant @CAPS15 one by one i hear the balloons popping @CAPS15 soon i was the last one left @CAPS15 i was gonna give up @CAPS5 i am @CAPS14 getting chased @CAPS15 chased until i am completely out of breath @CAPS15 @CAPS6 one of my friends @ORGANIZATION1 comes up in front of me @CAPS15 i @CAPS14 said "I don\'t care @CAPS14 get me out i @CAPS14 wanna lay down." @CAPS5 he took the balloon @CAPS15 @CAPS14 popped it @CAPS15 i asked "@CAPS11 i take a nap?" @CAPS15 he said "@CAPS12 because we have to get ready for the next round @CAPS5 you better rest up quick or you\'ll be dogging." @CAPS5 i @CAPS14 rested up as much as i could before we got everyone straped up for the next game i was hoping to get some kind of reward for winning the very first game of @CAPS4 but @CAPS12 my cousin @PERSON1 had to be a samrt alic about it @CAPS15 say" @CAPS14 getting to play the game isn\'t a big enough reward for you stingy!"@CAPS15 as always everyone starts @CAPS14 busting up laughing at every single little thing that my cousin @PERSON1 does but i do have to admit that he is one of the funniest guys that i have ever met.'.split( )

In [185]:
len(l)

435